In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [34]:
!pip install streamlit


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import streamlit as st
import os

# Assume the previous cells with AgentState, TalentScout, Onboarder, PolicyQA,
# build_user_driven_app, etc., have been executed and their definitions are available.

# --- Streamlit App ---

st.set_page_config(page_title="HR Automation Suite", layout="wide")

st.title("🤖 HR Automation Suite")

# Initialize session state for the workflow and its state
if 'workflow' not in st.session_state:
    # Assuming build_user_driven_app is defined in a previous cell
    st.session_state.workflow = build_user_driven_app()

if 'current_state' not in st.session_state:
    st.session_state.current_state = AgentState(
        input="",
        chat_history=[],
        job_requirements="",
        processed_resumes=[],
        best_candidate={},
        onboarding_plan="",
        final_output="",
        next_action="ExecuteTool", # Initial action doesn't matter much in user-driven
        resume_directory="",
        policy_file_path=""
    )

# --- Input Fields ---

st.header("Configuration")
policy_file = st.text_input("Enter the path to your company policies file:",
                            value=st.session_state.current_state.get('policy_file_path', ''))
if policy_file and os.path.isfile(policy_file):
    st.session_state.current_state['policy_file_path'] = policy_file
elif policy_file and not os.path.isfile(policy_file):
    st.error("Invalid policy file path.")


# --- Tool Selection and Execution ---

st.header("Run HR Tools")

tool_choice = st.radio("Choose a tool to run:",
                       ('TalentScout (Screen Resumes)',
                        'Onboarder (Generate Onboarding Plan)',
                        'PolicyQA (Answer Policy Questions)'))

run_button = st.button("Run Selected Tool")

if run_button:
    st.session_state.current_state['final_output'] = "" # Clear previous output

    if tool_choice == 'TalentScout (Screen Resumes)':
        st.session_state.current_state['next_action'] = "TalentScout"
        st.session_state.current_state['resume_directory'] = st.text_input("Enter the path to your resume folder:",
                                                                        value=st.session_state.current_state.get('resume_directory', ''))
        st.session_state.current_state['job_requirements'] = st.text_area("Enter the Job Requirements:",
                                                                       value=st.session_state.current_state.get('job_requirements', ''))

        if not st.session_state.current_state['resume_directory'] or not os.path.isdir(st.session_state.current_state['resume_directory']):
            st.error("Please enter a valid resume folder path.")
            run_button = False # Prevent execution
        elif not st.session_state.current_state['job_requirements']:
             st.error("Please enter the job requirements.")
             run_button = False # Prevent execution


    elif tool_choice == 'Onboarder (Generate Onboarding Plan)':
        st.session_state.current_state['next_action'] = "Onboarder"
        if not st.session_state.current_state.get("best_candidate"):
            st.warning("Please run TalentScout first to select a candidate.")
            run_button = False # Prevent execution

    elif tool_choice == 'PolicyQA (Answer Policy Questions)':
        st.session_state.current_state['next_action'] = "PolicyQA"
        st.session_state.current_state['input'] = st.text_input("Enter your policy question:")
        if not st.session_state.current_state['input']:
             st.error("Please enter a policy question.")
             run_button = False # Prevent execution


    if run_button: # Proceed if no validation errors
        with st.spinner(f"Running {st.session_state.current_state['next_action']}..."):
            # Execute the workflow for the selected action
            # Need to pass the state to invoke and update the session state
            updated_state = st.session_state.workflow.invoke(st.session_state.current_state)
            st.session_state.current_state.update(updated_state) # Update session state with results


# --- Display Output ---

st.header("Output")

if st.session_state.current_state.get('final_output'):
    st.markdown(st.session_state.current_state['final_output'])

if st.session_state.current_state.get('best_candidate'):
    st.subheader("Best Candidate Selected:")
    st.json(st.session_state.current_state['best_candidate'])

if st.session_state.current_state.get('processed_resumes'):
    st.subheader("Processed Resumes:")
    st.json(st.session_state.current_state['processed_resumes'])

if st.session_state.current_state.get('onboarding_plan'):
     st.subheader("Generated Onboarding Plan:")
     st.markdown(st.session_state.current_state['onboarding_plan'])

st.subheader("Current State (for debugging):")
st.json(st.session_state.current_state)

2025-10-04 20:38:22.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 20:38:22.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

To run this Streamlit app, you will need to:
1. Ensure all previous cells defining the workflow, agents, and tools have been executed successfully.
2. Save the code in the cell above as a Python file (e.g., `app.py`) in your Colab environment.
3. Open a new terminal in Colab (Terminal icon in the left sidebar).
4. Run the command: `streamlit run app.py`
5. Click the external URL provided by Streamlit to view your app.

In [36]:
!pip install spacy pdfplumber python-docx langchain langchain-google-genai langgraph pydantic


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 5.6 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 7.3 MB/s eta 0:00:02
     -------------- ------------------------- 4.7/12.8 MB 7.7 MB/s eta 0:00:02
     -------------------- ------------------- 6.6/12.8 MB 7.9 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 8.3 MB/s eta 0:00:01
     --------------------------------- ------ 10.7/12.8 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 8.9 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import re
from typing import TypedDict, Annotated, List, Dict
import operator
import time
import pdfplumber
import docx
import spacy
from spacy.matcher import Matcher
from typing import Dict, List
from pydantic import BaseModel
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
POLICY_FILE = "/content/Company_Policies.txt"

class ParsedResume(BaseModel):
    """Schema for the data extracted from a resume."""
    name: str
    email: str
    mobile_number: str
    skills: List[str]
    education: str
    work_experience_years: float
    filename: str


class SimulatedResumeParser:
    def __init__(self, resume_path: str, skills_file: str = None, custom_regex: str = None):
        self.resume_path = resume_path
        self.__skills_file = skills_file
        self.__custom_regex = custom_regex
        self.__text = self.__extract_text(resume_path)

        # Load spaCy NLP
        self.__nlp = spacy.load("en_core_web_sm")
        self.__doc = self.__nlp(self.__text)
        self.__matcher = Matcher(self.__nlp.vocab)

        # Extract details
        self.__details = self.__get_basic_details()

    def __extract_text(self, resume_path: str) -> str:
        """Extract text from TXT, PDF, or DOCX files."""
        ext = os.path.splitext(resume_path)[1].lower()

        if ext == ".pdf":
            text = ""
            try:
                with pdfplumber.open(resume_path) as pdf:
                    for page in pdf.pages:
                        text += page.extract_text() or ""
                return text
            except Exception as e:
                print(f"Error reading PDF {resume_path}: {e}")
                return ""

        elif ext == ".docx":
            text = ""
            try:
                doc = docx.Document(resume_path)
                for para in doc.paragraphs:
                    text += para.text + "\n"
                return text
            except Exception as e:
                print(f"Error reading DOCX {resume_path}: {e}")
                return ""

        else:  # fallback for plain text
            try:
                with open(resume_path, 'r', encoding="utf-8", errors="ignore") as f:
                    return f.read()
            except Exception as e:
                print(f"Error reading text file {resume_path}: {e}")
                return ""

    def __get_basic_details(self) -> Dict:
        """Extract details using spaCy NLP + regex (like original ResumeParser)."""
        text = self.__text
        doc = self.__doc
        details = {}

        name = None
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                name = ent.text
                break
        details["name"] = name if name else "Unknown Candidate"

        email = re.search(r"[\w\.-]+@[\w\.-]+", text)
        details["email"] = email.group(0) if email else "N/A"

        mobile = re.search(self.__custom_regex or r"(\+?\d[\d\-\s]{8,}\d)", text)
        details["mobile_number"] = mobile.group(0) if mobile else "N/A"

        skill_keywords = []
        if self.__skills_file and os.path.exists(self.__skills_file):
            with open(self.__skills_file, "r", encoding="utf-8") as f:
                skill_keywords = [line.strip() for line in f if line.strip()]
        else:
            skill_keywords = ["Python", "SQL", "Spark", "AWS", "Kubernetes",
                              "SEO", "Marketing", "JavaScript", "React", "Node.js"]

        found_skills = [s for s in skill_keywords if re.search(r"\b" + re.escape(s) + r"\b", text, re.IGNORECASE)]
        details["skills"] = found_skills

        edu_keywords = ["B.S.", "B.Sc", "M.S.", "M.Sc", "PhD", "Bachelor", "Master", "MBA", "B.Tech", "M.Tech"]
        education = None
        for token in doc:
            for kw in edu_keywords:
                if kw.lower() in token.text.lower():
                    education = token.sent.text
                    break
        if not education:
            education = "Degree Placeholder"
        details["education"] = education

        years_exp = 0.0
        exp_match = re.search(r"(\d+)\s+years", text, re.IGNORECASE)
        if exp_match:
            years_exp = float(exp_match.group(1))
        else:
            years_exp = 1.0
        details["work_experience_years"] = years_exp

        details["filename"] = os.path.basename(self.resume_path)
        return details

    def get_extracted_data(self) -> ParsedResume:
        """Return extracted resume data as a Pydantic model."""
        return ParsedResume(**self.__details)

In [39]:
import os
import pprint

if __name__ == "__main__":
    resume_folder = "content/Resume"
    skills_file = "content/Skills.txt"

    # 🔹 Check skills file
    if os.path.exists(skills_file):
        print(f"[OK] Skills file found: {skills_file}")
        with open(skills_file, "r") as f:
            print("Sample skills loaded:", [line.strip() for line in f.readlines()[:5]])
    else:
        print(f"[ERROR] Skills file not found: {skills_file}")

    # 🔹 Check resumes folder
    if not os.path.exists(resume_folder):
        print(f"[ERROR] Resume folder not found: {resume_folder}")
    else:
        print(f"[OK] Resume folder found: {resume_folder}")
        print("Files inside:", os.listdir(resume_folder))

    results = []
    for filename in os.listdir(resume_folder):
        filepath = os.path.join(resume_folder, filename)

        if not os.path.isfile(filepath):
            continue

        print(f"\n🔹 Parsing file: {filepath}")   # debug print
        parser = SimulatedResumeParser(filepath, skills_file=skills_file)
        extracted = parser.get_extracted_data()
        results.append(extracted.dict())

    print("\n✅ Final extracted results:")
    pprint.pprint(results)


[ERROR] Skills file not found: content/Skills.txt
[OK] Resume folder found: content/Resume
Files inside: ['Pratham_Thakkar.pdf', 'Praveen_kumar_T_resume.pdf', 'Vidit_Jain.pdf']

🔹 Parsing file: content/Resume\Pratham_Thakkar.pdf

🔹 Parsing file: content/Resume\Praveen_kumar_T_resume.pdf

🔹 Parsing file: content/Resume\Vidit_Jain.pdf

✅ Final extracted results:
[{'education': 'Implementedrobustfeaturesincludinguserauthenticationand • '
               'Rated1961(CandidateMaster,top200In-\n'
               'registration, post management, page creation, follower system, '
               'and dia)onCodeforces(Handle:ppt1524)and\n'
               'moderatorprivileges,etc. 5staronCodechef(Handle:ppt1524)\n'
               '• Techstackused:ReactJS|ExpressJS|NodeJS|MongoDB|Nginx •',
  'email': 'prathampthakkar@gmail.com',
  'filename': 'Pratham_Thakkar.pdf',
  'mobile_number': '+918849917720',
  'name': 'batch',
  'skills': ['Python', 'JavaScript'],
  'work_experience_years': 1.0},
 {'education

In [40]:
@tool
def generate_onboarding_plan(candidate_data: dict, role: str) -> str:
    """
    Uses an LLM to create a structured 30-60-90 day onboarding plan
    based on the candidate's skills and the specific job role.
    """
    prompt = f"""
    Create a detailed, actionable 30-60-90 day onboarding plan for the new hire:
    - Name: {candidate_data.get('name')}
    - Skills: {', '.join(candidate_data.get('skills', []))}
    - Target Role: {role}

    Structure the output clearly with the 30, 60, and 90-day milestones.
    """

    response = llm.invoke(prompt)
    return response.content

In [41]:
@tool
def process_and_score_resume(resume_filepath: str, job_requirements: str) -> dict:
    """
    Parses a single resume file, extracts key details using the parser,
    and then calculates a similarity score against the job_requirements using LLM.
    """
    print(f"  > Processing {os.path.basename(resume_filepath)}...")

    # 1. Use the simulated parser
    parser = SimulatedResumeParser(resume_filepath)
    parsed_data = parser.get_extracted_data().dict()

    # 2. Use LLM to score the relevance
    prompt = f"""
    You are a professional HR screener. Score the candidate's relevance for the job
    based on their extracted skills and experience.

    JOB REQUIREMENTS: {job_requirements}
    CANDIDATE PROFILE:
    - Skills: {', '.join(parsed_data['skills'])}
    - Experience: {parsed_data['work_experience_years']} years.
    - Education: {parsed_data['education']}

    Provide ONLY a single float score between 0.0 (not relevant) and 1.0 (perfect fit)
    in your response. Do not include any other text or explanation.
    """

    score = 0.5 # Default score if LLM extraction fails

    try:
        response = llm.invoke(prompt)
        # Attempt to find the score float in the response content
        score_match = re.search(r'(\d\.\d+)', response.content.strip())
        if score_match:
            score = float(score_match.group(1))
    except Exception as e:
        print(f"LLM scoring failed, using default: {e}")

    time.sleep(0.1) # Simulate complex operation time

    return {
        "candidate_data": parsed_data,
        "relevance_score": score,
        "filename": os.path.basename(resume_filepath)
    }

In [42]:
@tool
def retrieve_policy_answer(question: str, policy_file_path: str) -> str:
    """
    Simulates a RAG tool by searching a local policy file for an answer.
    """
    print(f"  > Searching policy documents for: '{question}'")

    try:
        with open(policy_file_path, 'r') as f:
            policy_text = f.read()
    except FileNotFoundError:
        return f"Error: Policy documents ({policy_file_path}) were not found. Cannot answer policy questions."

    # Use LLM to perform Q/A over the retrieved text (simulated RAG context)
    prompt = f"""
    Using ONLY the following policy text, answer the user's question concisely.
    If the answer is not available in the text, state that.

    POLICY TEXT:
    ---
    {policy_text}
    ---

    QUESTION: {question}
    """

    response = llm.invoke(prompt)
    return response.content

In [43]:
class AgentState(TypedDict):
    """Represents the state of our multi-agent system."""
    input: str
    chat_history: Annotated[List[HumanMessage], operator.add]
    job_requirements: str
    processed_resumes: List[dict]
    best_candidate: dict
    onboarding_plan: str
    final_output: str
    next_action: str
    resume_directory: str
    policy_file_path: str # Added policy file path to state


class ManagerOrchestrator:
    """Directs the flow based on the user's request."""

    def __init__(self, llm):
        self.tool_map = {
            "process_and_score_resume": "TalentScout",
            "retrieve_policy_answer": "PolicyQA",
            "generate_onboarding_plan": "Onboarder"
        }
        self.llm = llm.bind_tools([process_and_score_resume, retrieve_policy_answer, generate_onboarding_plan])

    def route_request(self, state: AgentState):
        """Routes the request to the appropriate agent."""

        if state.get('next_action') == "ONBOARD_BEST_CANDIDATE":
            return {"next_action": "Onboarder"}

        prompt = f"""
        You are the Manager/Orchestrator. Determine the user's intent:
        1. **RESUME SCREENING**: If the user asks to "find the best candidate" or "screen resumes" (delegate to TalentScout).
        2. **POLICY QUESTION**: If the user asks about "policy," "vacation," "leave," "expense" (delegate to PolicyQA).
        3. **ONBOARDING**: If the user asks to "create plan" or "onboard" and a candidate is already selected (delegate to Onboarder).

        Current Request: {state['input']}

        Delegate the task by calling the appropriate tool. If the request does not clearly fit, just output a conversational response.
        """

        try:
            # Use tool calling to force a decision
            response = self.llm.invoke(prompt)
            tool_call = response.tool_calls[0]
            tool_name = tool_call['name']

            agent_role = self.tool_map.get(tool_name)
            if agent_role:
                print(f"\n[Orchestrator] ROUTING: {tool_name} -> {agent_role}")
                return {"next_action": agent_role}

        except Exception:
            # If no tool call, assume conversational response is required
            print("\n[Orchestrator] ROUTING: Conversational Response/Fallback")
            return {"final_output": f"I'm an HR automation system. I can screen resumes, answer policy questions, and create onboarding plans. Please specify what you need."}


class TalentScout:
    """Agent for parsing, scoring, and selecting the best resume."""

    def run_screening(self, state: AgentState):
        """Scans the directory and runs the scoring tool for all files."""
        print(f"\n[TalentScout] Starting screening in directory: {state['resume_directory']}")

        # Filter files to only include common resume types
        resume_files = [os.path.join(state['resume_directory'], f)
                        for f in os.listdir(state['resume_directory'])
                        if f.lower().endswith(('.txt', '.pdf', '.docx'))]

        if not resume_files:
            return {"final_output": f"Error: No .txt, .pdf, or .docx files found in the specified directory: {state['resume_directory']}"}

        all_results = []
        for file_path in resume_files:
            try:
                # Invoke the tool for each resume
                result = process_and_score_resume.invoke({
                    "resume_filepath": file_path,
                    "job_requirements": state['job_requirements']
                })
                all_results.append(result)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

        # Select the best candidate
        best_candidate = max(all_results, key=lambda x: x['relevance_score'])

        output_message = (
            f"Screening complete! **{len(all_results)}** resumes processed.\n\n"
            f"**Top Candidate:** {best_candidate['candidate_data']['name']}\n"
            f"**File:** {best_candidate['filename']}\n"
            f"**Relevance Score:** {best_candidate['relevance_score']:.2f}/1.00\n"
            f"**Action:** The best candidate has been selected. Do you want to proceed with **Onboarding Plan** creation?"
        )

        return {
            "processed_resumes": all_results,
            "best_candidate": best_candidate['candidate_data'],
            "final_output": output_message,
            "next_action": "SCREENING_COMPLETE"
        }

class Onboarder:
    """Generate onboarding plan using LLM"""
    @staticmethod
    def create_plan(state: AgentState):
        candidate = state['best_candidate']
        if not candidate:
            return {"final_output": "No best candidate selected!", "next_action": ""}

        # Read company policies text
        try:
            with open(state['policy_file_path'], "r", encoding="utf-8") as f:
                policies_text = f.read()
        except Exception as e:
            return {"final_output": f"Error reading policy file: {e}", "next_action": ""}

        # LLM prompt
        prompt = f"""
You are an HR automation assistant. Generate a comprehensive onboarding plan
for a new employee based on the following information.

Candidate Details:
- Name: {candidate['name']}
- Email: {candidate['email']}
- Skills: {', '.join(candidate['skills'])}
- Education: {candidate['education']}
- Work Experience (years): {candidate['work_experience_years']}

Job Requirements:
{state['job_requirements']}

Company Policies (summarized or relevant sections):
{policies_text[:3000]}  # use first 3000 chars to avoid hitting token limits

Requirements:
- Provide a step-by-step onboarding plan.
- Include tasks such as document submission, orientation, setup, team introduction, and first project assignment.
- Reference company policies where relevant.
- Keep it professional and clear.

Output the onboarding plan in readable steps.
"""

        # Call the LLM
        try:
            from langchain_google_genai import ChatGoogleGenerativeAI
            llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)
            response = llm.invoke(prompt)
            plan_content = response.content # Access the content attribute
        except Exception as e:
            return {"final_output": f"Error generating onboarding plan with LLM: {e}", "next_action": ""}

        final_message = f"--- ✅ Onboarding Plan Generated ✅ ---\n{plan_content}"

        return {
            "onboarding_plan": plan_content,
            "final_output": final_message,
            "next_action": "PolicyQA"
        }


class PolicyQA:
    """Agent for answering policy questions using RAG."""

    def answer_question(self, state: AgentState):
        """Invokes the RAG tool to get the policy answer."""
        print(f"\n[PolicyQA] Answering question...")

        answer = retrieve_policy_answer.invoke({"question": state['input'], "policy_file_path": state['policy_file_path']}) # Pass policy file path

        final_message = f"--- 📄 **Policy Answer** 📄 ---\n{answer}"

        return {
            "final_output": final_message,
            "next_action": "QA_COMPLETE"
        }

def execute_tool_call(state: AgentState):
    """Executes the specific agent logic determined by the Manager."""
    action = state['next_action']

    if action == "TalentScout":
        return TalentScout().run_screening(state)
    elif action == "PolicyQA":
        return PolicyQA().answer_question(state)
    elif action == "Onboarder":
        return Onboarder().create_plan(state)

    return {"final_output": f"Error: Unknown tool or missing logic for action: {action}"}


def build_app():
    manager = ManagerOrchestrator(llm)
    workflow = StateGraph(AgentState)

    workflow.add_node("Manager", manager.route_request)
    workflow.add_node("ExecuteTool", execute_tool_call)

    workflow.set_entry_point("Manager")
    workflow.add_edge("Manager", "ExecuteTool")

    def decide_next_step(state: AgentState):
        """Conditional edge to route based on the ExecuteTool result."""
        action = state['next_action']

        if action == "SCREENING_COMPLETE":
            return "Manager"
        elif action == "QA_COMPLETE" or action == "ONBOARDING_COMPLETE":
            return END
        else:
            return "Manager"

    workflow.add_conditional_edges(
        "ExecuteTool",
        decide_next_step,
        {"Manager": "Manager", END: END}
    )

    return workflow.compile()

In [44]:
from langgraph.graph import StateGraph, END

def execute_tool_call(state: AgentState):
    """Executes the specific agent logic based on user selection."""
    action = state['next_action']

    if action == "TalentScout":
        output = TalentScout().run_screening(state)
        state.update(output)
        print("\n" + state['final_output'])
    elif action == "Onboarder":
        if not state.get("best_candidate"):
            print("\nError: No TalentScout results available. Run TalentScout first.")
            return state
        output = Onboarder().create_plan(state)
        state.update(output)
        print("\n" + state['final_output'])
    elif action == "PolicyQA":
        question = input("\nEnter your policy question (or type 'exit' to go back): ")
        if question.lower() != "exit":
            state['input'] = question
            output = PolicyQA().answer_question(state)
            print("\n" + output['final_output'])
            state.update(output)
    else:
        print(f"\nUnknown action: {action}")

    return state

def build_user_driven_app():
    """Builds a workflow that executes a single tool based on user input and then ends."""
    workflow = StateGraph(AgentState)

    workflow.add_node("ExecuteTool", execute_tool_call)
    workflow.set_entry_point("ExecuteTool")
    workflow.add_edge("ExecuteTool", END) # Remove conditional edge, go directly to END

    return workflow.compile()

In [ ]:
# Initialize state
state = AgentState(
    input="",
    chat_history=[],
    job_requirements="Entry level Data Engineer needing Python experience",
    processed_resumes=[],
    best_candidate={},
    onboarding_plan="",
    final_output="",
    next_action="ExecuteTool",
    resume_directory="content/Resumes",
    policy_file_path="content/Company_Policies.txt"
)

# Build workflow
workflow = build_user_driven_app()

# Run interactively
while True:
    print("\nAvailable tools:\n1. TalentScout\n2. Onboarder\n3. PolicyQA\n4. Exit")
    choice = input("Which tool do you want to run? ").strip().lower()

    if choice in ["exit", "4"]:
        print("Exiting workflow.")
        break
    elif choice in ["talentscout", "1"]:
        state['next_action'] = "TalentScout"
    elif choice in ["onboarder", "2"]:
        state['next_action'] = "Onboarder"
    elif choice in ["policyqa", "3"]:
        state['next_action'] = "PolicyQA"
    else:
        print("Invalid choice, please try again.")
        continue

    state = workflow.invoke(state)


Available tools:
1. TalentScout
2. Onboarder
3. PolicyQA
4. Exit

[TalentScout] Starting screening in directory: content/Resumes
  > Processing Pratham_Thakkar.pdf...
  > Processing Praveen_kumar_T_resume.pdf...
  > Processing Vidit_Jain.pdf...

Screening complete! **3** resumes processed.

**Top Candidate:** Png
**File:** Praveen_kumar_T_resume.pdf
**Relevance Score:** 0.95/1.00
**Action:** The best candidate has been selected. Do you want to proceed with **Onboarding Plan** creation?

Available tools:
1. TalentScout
2. Onboarder
3. PolicyQA
4. Exit

--- ✅ Onboarding Plan Generated ✅ ---
Here is a comprehensive onboarding plan for Png, tailored to the provided candidate details and company policies.

---

**Onboarding Plan for Png - Entry Level Data Engineer**

**Candidate Details:**
*   **Name:** Png
*   **Email:** praveenkumart236@gmail.com
*   **Skills:** Python, AWS
*   **Education:** B.Tech from IIT Jodhpur (2015-2019)
*   **Work Experience:** 1.0 years
*   **Job Role:** Entry L

In [ ]:
state = AgentState(
    input="",
    chat_history=[],
    job_requirements="",
    processed_resumes=[],
    best_candidate={},
    onboarding_plan="",
    final_output="",
    next_action="ExecuteTool",
    resume_directory="",
    policy_file_path=""
)

while True:
    policy_file = input("Enter the path to your company policies file: ").strip()
    if policy_file and os.path.isfile(policy_file):
        state['policy_file_path'] = policy_file
        break
    else:
        print("Invalid file path. Please try again.")

workflow = build_user_driven_app()

while True:
    print("\nAvailable tools:\n1. TalentScout\n2. Onboarder\n3. PolicyQA\n4. Exit")
    choice = input("Which tool do you want to run? ").strip().lower()

    if choice in ["exit", "4"]:
        print("Exiting workflow.")
        break
    elif choice in ["talentscout", "1"]:
        while True:
            resume_folder = input("Enter the path to your resume folder: ").strip()
            if resume_folder and os.path.isdir(resume_folder):
                state['resume_directory'] = resume_folder
                break
            else:
                print("Invalid folder path. Please try again.")

        state['job_requirements'] = input("Enter the Job Requirements: ").strip()
        state['next_action'] = "TalentScout"

    elif choice in ["onboarder", "2"]:
        if not state.get("best_candidate"):
            print("No TalentScout results yet. Run TalentScout first.")
            continue
        state['next_action'] = "Onboarder"

    elif choice in ["policyqa", "3"]:
        state['next_action'] = "PolicyQA"

    else:
        print("Invalid choice, please try again.")
        continue

    state = workflow.invoke(state)


Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.
Invalid file path. Please try again.


KeyboardInterrupt: Interrupted by user